# Prompt Engineering

### Simple code to impliment togather API

In [ ]:
from together import Together

client = Together(api_key="")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
    max_tokens=None,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>","<|eom_id|>"],
    stream=True
)

for token in response:
    if hasattr(token, 'choices'):
        print(token.choices[0].delta.content, end='', flush=True)

The city that never sleeps! New York has endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:

**Iconic Landmarks:**

1. Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.
2. Central Park: Explore the 843-acre park, which offers walking paths, lakes, restaurants, and plenty of people-watching opportunities.
3. Times Square: Experience the bright lights, giant billboards, and lively atmosphere of the "Crossroads of the World."
4. Brooklyn Bridge: Walk or bike across the iconic bridge for spectacular city views.
5. Empire State Building: Enjoy panoramic views of the city from the observation deck on the 86th floor.

**Museums and Galleries:**

1. The Metropolitan Museum of Art: One of the world's largest and most renowned museums, with a collection that spans over 5,000 years of human history.
2. American Museum of Natural History: Explore exh

## Implimentation of Togather API with Gradio

In [ ]:
import gradio as gr
from together import Together
import os
import time

# Initialize the Together client
# You should use environment variables for API keys in production
api_key = ""  # In production, use os.environ.get("TOGETHER_API_KEY")
client = Together(api_key=api_key)

def generate_response(role, prompt, temperature=0.7, top_p=0.7, top_k=50, max_tokens=1024, model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"):
    """Generate a response from the Together API"""
    try:
        # Create a completion using the Together API
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": role, "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=1,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=False  # We'll set this to False for the Gradio interface
        )
        
        # Extract and return the response content
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def stream_response(role, prompt, temperature=0.7, top_p=0.7, top_k=50, max_tokens=1024, model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"):
    """Stream a response from the Together API"""
    try:
        # Create a streaming completion
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": role, "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=1,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=True
        )
        
        # Stream the response token by token
        full_response = ""
        for token in response:
            if hasattr(token, 'choices') and token.choices[0].delta.content is not None:
                chunk = token.choices[0].delta.content
                full_response += chunk
                # Yield the updated response
                yield full_response
                time.sleep(0.01)  # Small delay for readability
        
        return full_response
    except Exception as e:
        return f"Error: {str(e)}"

# Function to handle flagging
def flag_interaction(role, prompt, response, model, temperature, top_p, top_k, max_tokens):
    """Save flagged interaction to CSV file"""
    import csv
    import datetime
    
    # Create a timestamp
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Prepare the data
    data = [timestamp, role, prompt, response, model, temperature, top_p, top_k, max_tokens]
    
    # Write to CSV file
    try:
        file_exists = os.path.isfile('flagged_interactions.csv')
        with open('flagged_interactions.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['Timestamp', 'User Role', 'Prompt', 'Response', 'Model', 'Temperature', 'Top P', 'Top K', 'Max Tokens'])
            writer.writerow(data)
        return "Interaction flagged successfully!"
    except Exception as e:
        return f"Error flagging interaction: {str(e)}"

# Define available models
models = [
    "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    "meta-llama/Llama-Vision-Free",
]

# Define available roles
roles = ["user", "system", "assistant"]

# Create the Gradio interface
with gr.Blocks(title="Together AI Chat Interface") as demo:
    gr.Markdown("# Together AI Chat Interface")
    gr.Markdown("Interact with language models through the Together AI API")
    
    with gr.Row():
        with gr.Column(scale=4):
            role_dropdown = gr.Dropdown(
                label="User Role",
                choices=roles,
                value="user"
            )
            prompt_input = gr.Textbox(
                label="Your Prompt",
                placeholder="Enter your prompt here...",
                lines=4
            )
        
        with gr.Column(scale=1):
            with gr.Accordion("Model Parameters", open=False):
                model_dropdown = gr.Dropdown(
                    label="Model",
                    choices=models,
                    value=models[0]
                )
                temperature_slider = gr.Slider(
                    label="Temperature",
                    minimum=0.1,
                    maximum=1.0,
                    value=0.7,
                    step=0.1
                )
                top_p_slider = gr.Slider(
                    label="Top P",
                    minimum=0.1,
                    maximum=1.0,
                    value=0.7,
                    step=0.1
                )
                top_k_slider = gr.Slider(
                    label="Top K",
                    minimum=1,
                    maximum=100,
                    value=50,
                    step=1
                )
                max_tokens_slider = gr.Slider(
                    label="Max Tokens",
                    minimum=64,
                    maximum=4096,
                    value=1024,
                    step=64
                )
                
    with gr.Row():
        submit_btn = gr.Button("Generate Response")
        stream_btn = gr.Button("Stream Response")
        clear_btn = gr.Button("Clear")
    
    output_area = gr.Textbox(
        label="Response",
        lines=10,
        show_copy_button=True
    )
    
    # Add a flag button
    flag_btn = gr.Button("Flag this interaction")
    flag_status = gr.Textbox(label="Flag Status", visible=True)
    
    # Set up event handlers
    submit_btn.click(
        fn=generate_response,
        inputs=[role_dropdown, prompt_input, temperature_slider, top_p_slider, top_k_slider, max_tokens_slider, model_dropdown],
        outputs=output_area
    )
    
    stream_btn.click(
        fn=stream_response,
        inputs=[role_dropdown, prompt_input, temperature_slider, top_p_slider, top_k_slider, max_tokens_slider, model_dropdown],
        outputs=output_area
    )
    
    clear_btn.click(
        fn=lambda: ("user", "", ""),
        inputs=None,
        outputs=[role_dropdown, prompt_input, output_area]
    )
    
    # Set up flag button handler
    flag_btn.click(
        fn=flag_interaction,
        inputs=[
            role_dropdown,
            prompt_input,
            output_area,
            model_dropdown,
            temperature_slider,
            top_p_slider,
            top_k_slider,
            max_tokens_slider
        ],
        outputs=flag_status
    )
    
    # Some example prompts
    example_prompts = [
        ["user", "What are some fun things to do in New York?"],
        ["user", "Explain quantum computing to a 10-year-old."],
        ["system", "You are a helpful AI assistant who speaks like a pirate."],
        ["user", "What are the main differences between Python and JavaScript?"]
    ]
    
    gr.Examples(
        examples=example_prompts,
        inputs=[role_dropdown, prompt_input]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(share=True, server_name="0.0.0.0")  # share=True creates a public link

* Running on local URL:  http://0.0.0.0:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
